## `ai_query` and Serving Endpoint

### Setup
You will need to run this in the 4.2 Demo Notebook:


```


telco_df.write.format("delta").saveAsTable("dbacademy.labuser10531593_1755786160.telco_input")


```

In [0]:
USE CATALOG IDENTIFIER(:catalog);
USE SCHEMA IDENTIFIER(:schema);

In [0]:
Select
ai_query
(
  'ML_AS_03_Demo4_Custom_labuser10531593_1755786160'
  ,
    STRUCT(
      SeniorCitizen
      ,tenure
      ,MonthlyCharges
      ,TotalCharges
      )
)
from telco_input


In [0]:
%python

model_name=dbutils.widgets.get('catalog')\
    +'.'+dbutils.widgets.get('schema')\
    +'.'+dbutils.widgets.get('model')

print(model_name)

In [0]:
%python
import mlflow
model_uri = f"models:/{model_name}/1"
pyfunc_udf = mlflow.pyfunc.spark_udf(spark, model_uri)
spark.udf.register("telco_model_apply", pyfunc_udf)

In [0]:

WITH base
AS
(
  SELECT
  *,
  telco_model_apply
  (
      STRUCT(
        SeniorCitizen
        ,tenure
        ,MonthlyCharges
        ,TotalCharges
        )
  ) as predictions
  from telco_input
)
SELECT 
    predictions.prob_No
  , predictions.prob_Yes
  , predictions.prediction
  , *
  EXCEPT (predictions) 
FROM base